In [3]:
import paramiko

# SSH 접속 정보
ip = "172.16.16.24"
user = "root"
pw = "asd123!@"

def cc(command):
    """SSH로 명령을 실행하고 결과를 출력"""
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(command)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    if err:
        print("⚠️ 오류 발생:", err)
    return out.strip()


def dhcp_install():
    """DHCP 서버 설치"""
    cmd = """
if [ "$(id -u)" -ne 0 ]; then
    echo "이 스크립트는 root 권한으로 실행해야 합니다."
    exit 1
fi
DEBIAN_FRONTEND=noninteractive apt update -y
DEBIAN_FRONTEND=noninteractive apt install isc-dhcp-server -y
"""
    cc(cmd)
    print("DHCP 서버 설치 완료")


def set_dhcp_file():
    """DHCP 설정 파일 구성"""
    # /etc/default/isc-dhcp-server 확인
    print(cc("cat /etc/default/isc-dhcp-server"))
    
    # 사용자 입력
    NI = input("위에서 확인한 NIC 이름을 입력해주세요 ex. ens33: ")
    print(cc("ip a"))
    subnet = input("위에서 확인한 subnet을 입력하세요 ex. 192.168.0.0: ")
    netmask = input("위에서 확인한 netmask을 입력하세요 ex. 255.255.255.0: ")
    exclusion_start = input("제외할 ip 시작 주소를 적으세요 ex. 192.168.0.100: ")
    exclusion_end = input("제외할 ip 마지막 주소를 적으세요 ex. 192.168.0.200: ")
    gateway = input("게이트웨이를 입력하세요 ex. 192.168.0.1: ")
    domain = input("도메인 서버를 입력하세요 ex. 8.8.8.8: ")
    pool = input("pool의 이름을 지정하세요 ex. 'example.local': ")

    # NIC 업데이트
    ch_NI_cmd = f"""
sed -i 's|^INTERFACESv4=.*$|INTERFACESv4="{NI}"|' /etc/default/isc-dhcp-server
"""
    cc(ch_NI_cmd)
    print("/etc/default/isc-dhcp-server 파일을 해당 NI로 변경했습니다.")

    # dhcpd.conf 업데이트
    add_dhcp_file_cmd = f"""
echo '
subnet {subnet} netmask {netmask} {{
    range {exclusion_start} {exclusion_end};
    option routers {gateway};
    option subnet-mask {netmask};
    option domain-name-servers {domain}, 8.8.8.8;
    option domain-name "{pool}";
}}
' >> /etc/dhcp/dhcpd.conf
"""
    cc(add_dhcp_file_cmd)
    print("/etc/dhcp/dhcpd.conf 파일 업데이트 완료")


def dhcp_request():
    """DHCP 요청 및 인터페이스 확인"""
    cmd = """
apt install isc-dhcp-client -y
dhclient -v
ip a
systemctl restart isc-dhcp-server
systemctl enable isc-dhcp-server
systemctl status isc-dhcp-server
"""
    cc(cmd)
    print("DHCP 요청 완료 및 서버 실행 확인")


def go():
 #   dhcp_install()
 #   set_dhcp_file()
    dhcp_request()
    print("모든 작업 완료")


if __name__ == "__main__":
    go()


⚠️ 오류 발생: 

debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
Internet Systems Consortium DHCP Client 4.4.3-P1
Copyright 2004-2022 Internet Systems Consortium.
Al